BTEngineDark

GhostOkaamiii
Wolfrank Guzman 

12.12.2025
Parquet Version 


In [2]:
#Claudeversion
#CLAUDE VERSION 
#version 2 

# AlgoHaus Backtester v5.0 - Complete Dark Theme Edition with KPI Dashboard
# Features: Pure Dark Theme UI, Interactive KPI Dashboard with Charts, Advanced Metrics, AI Analysis
# Modified to read PARQUET files with comprehensive visualizations

import customtkinter as ctk
import tkinter as tk
from tkinter import messagebox, filedialog
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import webbrowser
import os
import tempfile
import inspect
import pathlib
import threading
import queue
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import re 
import logging

# Configure CustomTkinter - COMPLETE DARK THEME
ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("green")

# Configure basic logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# ======================================================================
# 1. MARGIN AND PIP CALCULATIONS
# ======================================================================
class ForexCalculator:
    """Handle all forex calculations including margins, pip values, and position sizing"""
   
    LEVERAGE_OPTIONS = [1, 10, 20, 30, 50, 100, 200, 500]
   
    PIP_VALUES = {
        'EUR/USD': 0.0001, 'GBP/USD': 0.0001, 'USD/JPY': 0.01,
        'USD/CHF': 0.0001, 'USD/CAD': 0.0001, 'AUD/USD': 0.0001,
        'NZD/USD': 0.0001, 'EUR/JPY': 0.01, 'GBP/JPY': 0.01,
        'AUD/JPY': 0.01, 'EUR/GBP': 0.0001, 'EUR/CHF': 0.0001,
        'GBP/CHF': 0.0001, 'CHF/JPY': 0.01, 'CAD/JPY': 0.01
    }
    
    @staticmethod
    def get_max_units_per_100usd(leverage=50):
        return 100 * leverage 
   
    @staticmethod
    def calculate_pip_value_in_usd(pair, unit_size, current_price, conversion_rate=1.0):
        pip_size = ForexCalculator.PIP_VALUES.get(pair, 0.0001) 
        
        if pair.endswith('/USD'):
            pip_value = pip_size * unit_size
        elif pair.startswith('USD/'):
            pip_value = (pip_size / current_price) * unit_size
        else:
            pip_value_in_base = pip_size * unit_size
            pip_value = pip_value_in_base * conversion_rate 
        
        return pip_value
   
    @staticmethod
    def calculate_margin_required(pair, unit_size, current_price, leverage, conversion_rate=1.0):
        if pair.endswith('/USD'):
            position_value = unit_size * current_price
        elif pair.startswith('USD/'):
            position_value = unit_size
        else:
            position_value = unit_size * conversion_rate 
        
        margin_required = position_value / leverage
        return margin_required

# ======================================================================
# 2. DATA LOADING FUNCTIONS
# ======================================================================
def load_pair_data(pair_name: str, base_folder: pathlib.Path, start_date: datetime, end_date: datetime, timeframe: str):
    pair_clean = pair_name.replace('/', '_')
    subfolder = base_folder / pair_clean
    parquet_files = []
   
    if subfolder.is_dir():
        parquet_files = list(subfolder.glob("*.parquet"))
   
    if not parquet_files:
        parquet_files = [f for f in base_folder.glob("*.parquet") if pair_clean.lower() in f.name.lower()]
   
    if not parquet_files:
        raise FileNotFoundError(f"No PARQUET file found for {pair_name}")
   
    parquet_path = parquet_files[0]
    df = pd.read_parquet(parquet_path, engine='pyarrow')
    
    if df.empty:
        raise ValueError("PARQUET file is empty")
   
    cols_lower = [c.strip().lower() for c in df.columns]
    col_map = {
        'datetime': ['datetime', 'date', 'time', 'timestamp', 'date_time', 'index'],
        'open': ['open', 'o', 'bid', 'ask', 'price'],
        'high': ['high', 'h', 'max'],
        'low': ['low', 'l', 'min'],
        'close': ['close', 'c', 'last', 'price'],
        'volume': ['volume', 'vol', 'v', 'tick_volume', 'size']
    }
   
    rename = {}
    for target, aliases in col_map.items():
        for alias in aliases:
            if any(alias in col for col in cols_lower):
                orig = next(col for col in df.columns if alias in col.lower())
                rename[orig] = target
                break
        else:
            if target != 'volume':
                raise KeyError(f"Column for '{target}' not found")
   
    df = df.rename(columns=rename)
    
    if 'volume' not in df.columns:
        df['volume'] = 1000
   
    for col in ['open', 'high', 'low', 'close', 'volume']:
        if col in df.columns:
            df[col] = df[col].astype('float32')
   
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce', utc=True)
    df = df.dropna(subset=['datetime'])
    df['datetime'] = df['datetime'].dt.tz_localize(None)
    
    df = df.drop_duplicates(subset='datetime', keep='first')
    df = df.sort_values('datetime')
   
    actual_start = df['datetime'].min().date()
    actual_end = df['datetime'].max().date()
   
    df = df.set_index('datetime')
    user_start = max(pd.Timestamp(start_date.date()), df.index.min())
    user_end = min(pd.Timestamp(end_date.date()) + pd.Timedelta(hours=23, minutes=59, seconds=59), df.index.max())
    df = df.loc[user_start:user_end].copy()
   
    if df.empty:
        raise ValueError(f"No data in range {start_date.date()} to {end_date.date()}")
   
    if timeframe != '1min':
        rule = {'5min': '5T', '15min': '15T', '1hr': '1H', '1Day': '1D'}.get(timeframe, '1T')
        df = df.resample(rule).agg({
            'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'
        }).dropna()
   
    df = df.reset_index()
    df['date'] = df['datetime'].dt.date
   
    daily = df.groupby('date').agg({
        'high': 'max', 'low': 'min', 'close': 'last'
    })
    daily.columns = ['day_high', 'day_low', 'day_close']
    daily['prev_high'] = daily['day_high'].shift(1)
    daily['prev_low'] = daily['day_low'].shift(1)
    daily['prev_close'] = daily['day_close'].shift(1)
   
    daily = daily.reset_index()
    df = pd.merge(df, daily[['date', 'prev_high', 'prev_low', 'prev_close']], on='date', how='left')
    df[['prev_high', 'prev_low', 'prev_close']] = df[['prev_high', 'prev_low', 'prev_close']].ffill()
   
    return df, actual_start, actual_end

# ======================================================================
# 3. TRADING STRATEGIES
# ======================================================================
class TradingStrategies:
    @staticmethod
    def vwap_crossover_strategy(df, sl_pips, tp_pips, pip_value):
        df = df.copy()
        df['tpv'] = df['volume'] * (df['high'] + df['low'] + df['close']) / 3
        df['cumvol'] = df.groupby('date')['volume'].cumsum()
        df['cumtpv'] = df.groupby('date')['tpv'].cumsum()
        df['vwap'] = df['cumtpv'] / df['cumvol']
        
        df['prev_close'] = df['close'].shift(1)
        df['prev_vwap'] = df['vwap'].shift(1)
        
        df['signal'] = np.where(
            (df['prev_close'] <= df['prev_vwap']) & (df['close'] > df['vwap']), 'BUY',
            np.where((df['prev_close'] >= df['prev_vwap']) & (df['close'] < df['vwap']), 'SELL', None)
        )
        
        entries = df[df['signal'].notna()].copy()
        trades = []
        
        for idx, row in entries.iterrows():
            remaining_data = df[df.index > idx].reset_index(drop=True)
            if len(remaining_data) > 0:
                trades.append({
                    'datetime': row['datetime'],
                    'entry_price': row['close'],
                    'signal': row['signal'],
                    'stop_loss': sl_pips * pip_value,
                    'take_profit': tp_pips * pip_value,
                    'day_data': remaining_data
                })
        
        return trades
    
    @staticmethod
    def opening_range_strategy(df, sl_pips, tp_pips, pip_value):
        df = df.copy()
        trades = []
        
        for date in df['date'].unique():
            day_data = df[df['date'] == date].reset_index(drop=True)
            if len(day_data) < 31: 
                continue
            
            opening_range = day_data.iloc[:30]
            or_high = opening_range['high'].max()
            or_low = opening_range['low'].min()
            or_mid = (or_high + or_low) / 2
            
            if len(day_data) > 30:
                entry_bar = day_data.iloc[30]
                signal = 'BUY' if entry_bar['close'] > or_mid else 'SELL'
                
                trades.append({
                    'datetime': entry_bar['datetime'],
                    'entry_price': entry_bar['close'],
                    'signal': signal,
                    'stop_loss': sl_pips * pip_value,
                    'take_profit': tp_pips * pip_value,
                    'day_data': day_data[31:].reset_index(drop=True)
                })
        
        return trades

# ======================================================================
# 4. ENHANCED BACKTESTER
# ======================================================================
class EnhancedBacktester:
    def __init__(self, df, initial_balance=10000, unit_size=10000, pip_value=0.0001, leverage=50):
        self.df = df
        self.initial_balance = initial_balance
        self.unit_size = unit_size
        self.pip_value = pip_value
        self.leverage = leverage
        self.results = None
        
    def run_backtest(self, strategy_func, sl_pips, tp_pips, pair_name):
        trades = strategy_func(self.df, sl_pips, tp_pips, self.pip_value)
        
        if not trades:
            self.results = pd.DataFrame()
            return "No trades generated.", {}
            
        results = []
        current_balance = self.initial_balance
        trade_number = 1
        
        for t in trades:
            entry_price = t['entry_price']
            signal = t['signal']
            bars = t['day_data']
            
            margin_required = ForexCalculator.calculate_margin_required(
                pair_name, self.unit_size, entry_price, self.leverage, 1.0
            )
            
            if margin_required > current_balance * 0.5:
                continue
            
            pip_value_usd = ForexCalculator.calculate_pip_value_in_usd(
                pair_name, self.unit_size, entry_price, 1.0
            )
            
            if signal == 'BUY':
                stop_level = entry_price - (sl_pips * self.pip_value)
                take_level = entry_price + (tp_pips * self.pip_value)
            else:
                stop_level = entry_price + (sl_pips * self.pip_value)
                take_level = entry_price - (tp_pips * self.pip_value)
            
            exit_idx = len(bars) - 1
            exit_reason = 'Timeout'
            exit_price = bars.iloc[-1]['close'] if len(bars) > 0 else entry_price
            
            for i, bar in bars.iterrows():
                if signal == 'BUY':
                    if bar['low'] <= stop_level:
                        exit_idx = i
                        exit_price = stop_level
                        exit_reason = 'SL'
                        break
                    elif bar['high'] >= take_level:
                        exit_idx = i
                        exit_price = take_level
                        exit_reason = 'TP'
                        break
                else:
                    if bar['high'] >= stop_level:
                        exit_idx = i
                        exit_price = stop_level
                        exit_reason = 'SL'
                        break
                    elif bar['low'] <= take_level:
                        exit_idx = i
                        exit_price = take_level
                        exit_reason = 'TP'
                        break
            
            if signal == 'BUY':
                pips_pnl = (exit_price - entry_price) / self.pip_value
            else:
                pips_pnl = (entry_price - exit_price) / self.pip_value
            
            monetary_pnl = pips_pnl * pip_value_usd
            entry_time = t['datetime']
            
            if exit_idx < len(bars):
                exit_time = bars.iloc[exit_idx]['datetime']
            else:
                exit_time = bars.iloc[-1]['datetime'] if len(bars) > 0 else entry_time
            
            time_in_trade = exit_time - entry_time
            hours_in_trade = time_in_trade.total_seconds() / 3600
            current_balance += monetary_pnl
            
            results.append({
                'trade_number': f"{trade_number:05d}",
                'entry_time': entry_time,
                'exit_time': exit_time,
                'time_in_trade_hours': round(hours_in_trade, 2),
                'signal': signal,
                'entry_price': entry_price,
                'exit_price': exit_price,
                'exit_reason': exit_reason,
                'pips_pnl': round(pips_pnl, 2),
                'monetary_pnl': round(monetary_pnl, 2),
                'unit_size': self.unit_size,
                'margin_used': round(margin_required, 2),
                'balance': round(current_balance, 2),
                'pip_value_usd': round(pip_value_usd, 4)
            })
            
            trade_number += 1
        
        self.results = pd.DataFrame(results)
        
        if not self.results.empty:
            total_pnl = self.results['monetary_pnl'].sum()
            total_pips = self.results['pips_pnl'].sum()
            win_rate = (self.results['pips_pnl'] > 0).mean() * 100
            
            summary = (f"TRADES: {len(self.results)}\n"
                      f"WIN RATE: {win_rate:.1f}%\n"
                      f"P&L: ${total_pnl:,.2f}\n"
                      f"PIPS: {total_pips:,.1f}\n"
                      f"FINAL BALANCE: ${current_balance:,.2f}")
            
            metrics = self.calculate_metrics()
        else:
            summary = "No trades executed"
            metrics = {}
        
        return summary, metrics
    
    def calculate_metrics(self):
        trades_df = self.results
        if trades_df.empty:
            return {}
        
        total_trades = len(trades_df)
        winning_trades = (trades_df['monetary_pnl'] > 0).sum()
        losing_trades = (trades_df['monetary_pnl'] < 0).sum()
        win_rate = (winning_trades / total_trades * 100) if total_trades > 0 else 0
        
        total_pnl = trades_df['monetary_pnl'].sum()
        total_pips = trades_df['pips_pnl'].sum()
        avg_win = trades_df[trades_df['monetary_pnl'] > 0]['monetary_pnl'].mean() if winning_trades > 0 else 0
        avg_loss = abs(trades_df[trades_df['monetary_pnl'] < 0]['monetary_pnl'].mean()) if losing_trades > 0 else 0
        
        gross_profit = trades_df[trades_df['monetary_pnl'] > 0]['monetary_pnl'].sum()
        gross_loss = abs(trades_df[trades_df['monetary_pnl'] < 0]['monetary_pnl'].sum())
        profit_factor = gross_profit / gross_loss if gross_loss > 0 else float('inf')
        
        final_balance = trades_df['balance'].iloc[-1] if not trades_df.empty else self.initial_balance
        total_return = ((final_balance - self.initial_balance) / self.initial_balance) * 100
        
        equity_curve = self.initial_balance + trades_df['monetary_pnl'].cumsum()
        cummax = equity_curve.expanding().max()
        drawdown = (equity_curve - cummax) / cummax
        max_drawdown_pct = drawdown.min() * 100
        
        if len(trades_df) > 1:
            daily_returns = trades_df.groupby(trades_df['entry_time'].dt.date)['monetary_pnl'].sum()
            daily_returns_pct = daily_returns / self.initial_balance
            sharpe = (daily_returns_pct.mean() * 252) / (daily_returns_pct.std() * np.sqrt(252)) if daily_returns_pct.std() > 0 else 0
            
            negative_returns = daily_returns_pct[daily_returns_pct < 0]
            downside_std = negative_returns.std() if len(negative_returns) > 0 else 0
            sortino = (daily_returns_pct.mean() * 252) / (downside_std * np.sqrt(252)) if downside_std > 0 else 0
        else:
            sharpe = 0
            sortino = 0
        
        return {
            'total_trades': total_trades,
            'winning_trades': winning_trades,
            'losing_trades': losing_trades,
            'win_rate_%': round(win_rate, 2),
            'total_pnl_$': round(total_pnl, 2),
            'total_pips': round(total_pips, 1),
            'avg_win_$': round(avg_win, 2),
            'avg_loss_$': round(avg_loss, 2),
            'profit_factor': round(profit_factor, 2),
            'max_drawdown_%': round(max_drawdown_pct, 2),
            'sharpe_ratio': round(sharpe, 2),
            'sortino_ratio': round(sortino, 2),
            'total_return_%': round(total_return, 2),
            'final_balance_$': round(final_balance, 2)
        }

# ======================================================================
# 5. KPI DASHBOARD GENERATOR
# ======================================================================
class KPIDashboard:
    @staticmethod
    def generate_kpi_charts(metrics, trades_df):
        charts_html = ""
        
        charts_html += KPIDashboard.create_gauge_charts(metrics)
        charts_html += KPIDashboard.create_pnl_pie_chart(trades_df)
        charts_html += KPIDashboard.create_performance_heatmap(trades_df)
        charts_html += KPIDashboard.create_risk_radar_chart(metrics)
        charts_html += KPIDashboard.create_monthly_performance_bar(trades_df)
        charts_html += KPIDashboard.create_drawdown_line_chart(trades_df, metrics)
        
        return charts_html
    
    @staticmethod
    def create_gauge_charts(metrics):
        kpis = {
            'Sharpe Ratio': {'value': metrics.get('sharpe_ratio', 0), 'min': -3, 'max': 3},
            'Sortino Ratio': {'value': metrics.get('sortino_ratio', 0), 'min': -3, 'max': 3},
            'Win Rate': {'value': metrics.get('win_rate_%', 0), 'min': 0, 'max': 100},
            'Profit Factor': {'value': metrics.get('profit_factor', 0), 'min': 0, 'max': 3}
        }
        
        html = '<div class="kpi-gauges-container">'
        
        for kpi_name, kpi_data in kpis.items():
            value = kpi_data['value']
            min_val = kpi_data['min']
            max_val = kpi_data['max']
            
            if kpi_name in ['Sharpe Ratio', 'Sortino Ratio']:
                color = '#ff4757' if value < 0 else '#32ff7e' if value > 1 else '#ff9f43'
            elif kpi_name == 'Win Rate':
                color = '#ff4757' if value < 40 else '#32ff7e' if value > 60 else '#ff9f43'
            elif kpi_name == 'Profit Factor':
                color = '#ff4757' if value < 1 else '#32ff7e' if value > 1.5 else '#ff9f43'
            else:
                color = '#00ff41'
            
            fig = go.Figure(go.Indicator(
                mode = "gauge+number",
                value = value,
                domain = {'x': [0, 1], 'y': [0, 1]},
                title = {'text': kpi_name, 'font': {'color': '#ffffff', 'size': 14}},
                number = {'font': {'color': color, 'size': 20}},
                gauge = {
                    'axis': {'range': [min_val, max_val], 'tickcolor': '#333333', 'tickfont': {'color': '#ffffff'}},
                    'bar': {'color': color},
                    'bgcolor': "#000000",
                    'borderwidth': 2,
                    'bordercolor': "#333333"
                }
            ))
            
            fig.update_layout(
                template="plotly_dark",
                paper_bgcolor='#000000',
                plot_bgcolor='#000000',
                font={'color': '#ffffff'},
                height=300,
                margin=dict(l=20, r=20, t=40, b=20)
            )
            
            html += f'<div class="kpi-gauge-box"><h4 class="kpi-title">{kpi_name}</h4>{fig.to_html(full_html=False, include_plotlyjs=False)}</div>'
        
        html += '</div>'
        return html
    
    @staticmethod
    def create_pnl_pie_chart(trades_df):
        if trades_df.empty:
            return ""
        
        wins = len(trades_df[trades_df['monetary_pnl'] > 0])
        losses = len(trades_df[trades_df['monetary_pnl'] < 0])
        
        fig = go.Figure(data=[go.Pie(
            labels=['Winning Trades', 'Losing Trades'], 
            values=[wins, losses],
            hole=0.4,
            marker=dict(colors=['#00ff41', '#ff4757'], line=dict(color='#000000', width=2))
        )])
        
        fig.update_layout(
            template="plotly_dark",
            title={'text': 'Trade Distribution', 'font': {'color': '#ffffff', 'size': 16}, 'x': 0.5},
            paper_bgcolor='#000000',
            plot_bgcolor='#000000',
            height=400
        )
        
        return f'<div class="chart-box"><h3>Trade Distribution</h3>{fig.to_html(full_html=False, include_plotlyjs=False)}</div>'
    
    @staticmethod
    def create_performance_heatmap(trades_df):
        if trades_df.empty or len(trades_df) < 10:
            return ""
        
        trades_df_copy = trades_df.copy()
        trades_df_copy['hour'] = trades_df_copy['entry_time'].dt.hour
        trades_df_copy['day_name'] = trades_df_copy['entry_time'].dt.day_name()
        
        heatmap_data = trades_df_copy.groupby(['day_name', 'hour'])['monetary_pnl'].sum().reset_index()
        pivot_data = heatmap_data.pivot(index='day_name', columns='hour', values='monetary_pnl').fillna(0)
        
        fig = go.Figure(data=go.Heatmap(
            z=pivot_data.values,
            x=pivot_data.columns,
            y=pivot_data.index,
            colorscale='RdYlGn',
            zmid=0
        ))
        
        fig.update_layout(
            template="plotly_dark",
            title={'text': 'Performance Heatmap', 'font': {'color': '#ffffff', 'size': 16}, 'x': 0.5},
            paper_bgcolor='#000000',
            plot_bgcolor='#000000',
            height=400
        )
        
        return f'<div class="chart-box"><h3>Performance Heatmap</h3>{fig.to_html(full_html=False, include_plotlyjs=False)}</div>'
    
    @staticmethod
    def create_risk_radar_chart(metrics):
        categories = ['Sharpe Ratio', 'Sortino Ratio', 'Profit Factor', 'Win Rate']
        values = [
            max(0, min(1, (metrics.get('sharpe_ratio', 0) + 3) / 6)),
            max(0, min(1, (metrics.get('sortino_ratio', 0) + 3) / 6)),
            max(0, min(1, metrics.get('profit_factor', 0) / 3)),
            max(0, min(1, metrics.get('win_rate_%', 0) / 100))
        ]
        
        fig = go.Figure()
        fig.add_trace(go.Scatterpolar(
            r=values,
            theta=categories,
            fill='toself',
            line=dict(color='#00ff41'),
            fillcolor='rgba(0, 255, 65, 0.3)'
        ))
        
        fig.update_layout(
            template="plotly_dark",
            title={'text': 'Risk Profile', 'font': {'color': '#ffffff', 'size': 16}, 'x': 0.5},
            paper_bgcolor='#000000',
            plot_bgcolor='#000000',
            height=400
        )
        
        return f'<div class="chart-box"><h3>Risk Profile</h3>{fig.to_html(full_html=False, include_plotlyjs=False)}</div>'
    
    @staticmethod
    def create_monthly_performance_bar(trades_df):
        if trades_df.empty or len(trades_df) < 5:
            return ""
        
        trades_df_copy = trades_df.copy()
        trades_df_copy['month'] = trades_df_copy['entry_time'].dt.to_period('M')
        monthly_pnl = trades_df_copy.groupby('month')['monetary_pnl'].sum()
        
        colors = ['#00ff41' if x >= 0 else '#ff4757' for x in monthly_pnl.values]
        
        fig = go.Figure(data=[go.Bar(
            x=[str(m) for m in monthly_pnl.index],
            y=monthly_pnl.values,
            marker_color=colors
        )])
        
        fig.update_layout(
            template="plotly_dark",
            title={'text': 'Monthly Performance', 'font': {'color': '#ffffff', 'size': 16}, 'x': 0.5},
            paper_bgcolor='#000000',
            plot_bgcolor='#000000',
            height=400
        )
        
        return f'<div class="chart-box"><h3>Monthly Performance</h3>{fig.to_html(full_html=False, include_plotlyjs=False)}</div>'
    
    @staticmethod
    def create_drawdown_line_chart(trades_df, metrics):
        if trades_df.empty:
            return ""
        
        initial_balance = trades_df['balance'].iloc[0] - trades_df['monetary_pnl'].iloc[0]
        equity_curve = initial_balance + trades_df['monetary_pnl'].cumsum()
        cummax = equity_curve.expanding().max()
        drawdown = (equity_curve - cummax) / cummax * 100
        
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=trades_df['exit_time'],
            y=drawdown,
            mode='lines',
            line=dict(color='#ff4757', width=2),
            fill='tozeroy',
            fillcolor='rgba(255, 71, 87, 0.3)'
        ))
        
        fig.update_layout(
            template="plotly_dark",
            title={'text': 'Drawdown Analysis', 'font': {'color': '#ffffff', 'size': 16}, 'x': 0.5},
            paper_bgcolor='#000000',
            plot_bgcolor='#000000',
            height=400
        )
        
        return f'<div class="chart-box"><h3>Drawdown Analysis</h3>{fig.to_html(full_html=False, include_plotlyjs=False)}</div>'

# ======================================================================
# 6. HTML REPORT GENERATOR
# ======================================================================
class HTMLReportGenerator:
    @staticmethod
    def generate_report(metrics, trades_df, strategy, timeframe, pair, initial_balance, unit_size, df=None):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        kpi_dashboard_html = KPIDashboard.generate_kpi_charts(metrics, trades_df)
        
        html = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>AlgoHaus Dark Report - {pair} {strategy.__name__}</title>
            <meta charset="utf-8">
            <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
            <style>
                * {{ box-sizing: border-box; margin: 0; padding: 0; }}
                body {{ font-family: 'Helvetica', monospace; background: #000000; color: #ffffff; line-height: 1.6; }}
                
                .header {{ background: #000000; color: #ffffff; padding: 30px; border-bottom: 2px solid #00ff41; text-align: center; }}
                .header h1 {{ font-size: 24px; margin-bottom: 10px; color: #00ff41; text-shadow: 0 0 10px rgba(0, 255, 65, 0.5); }}
                .header p {{ color: #888888; font-size: 14px; }}
                
                .kpi-dashboard-section {{ background: #000000; padding: 20px; border-bottom: 1px solid #222222; }}
                .kpi-dashboard-section h2 {{ color: #00ff41; font-size: 18px; margin-bottom: 20px; font-weight: bold; }}
                
                .kpi-gauges-container {{ display: grid; grid-template-columns: repeat(auto-fit, minmax(300px, 1fr)); gap: 20px; margin-bottom: 30px; }}
                .kpi-gauge-box {{ background: #0a0a0a; border: 1px solid #222222; border-radius: 8px; padding: 15px; transition: all 0.3s ease; }}
                .kpi-gauge-box:hover {{ background: #111111; border-color: #00ff41; box-shadow: 0 0 10px rgba(0, 255, 65, 0.2); }}
                .kpi-title {{ color: #00ff41; font-size: 14px; margin-bottom: 10px; text-align: center; font-weight: bold; }}
                
                .charts-container {{ background: #000000; padding: 20px; }}
                .charts-grid {{ display: grid; grid-template-columns: repeat(auto-fit, minmax(500px, 1fr)); gap: 20px; }}
                .chart-box {{ background: #0a0a0a; border: 1px solid #222222; border-radius: 8px; padding: 15px; }}
                .chart-box h3 {{ color: #00ff41; font-size: 14px; margin-bottom: 15px; text-align: center; }}
                
                .metrics-section {{ background: #000000; padding: 20px; border-bottom: 1px solid #222222; }}
                .metrics-grid {{ display: grid; grid-template-columns: repeat(auto-fit, minmax(250px, 1fr)); gap: 20px; margin-top: 20px; }}
                .metric-card {{ background: #0a0a0a; border: 1px solid #222222; border-radius: 8px; padding: 15px; }}
                .metric-label {{ color: #888888; font-size: 12px; text-transform: uppercase; margin-bottom: 5px; }}
                .metric-value {{ font-size: 24px; font-weight: bold; color: #ffffff; }}
                .metric-value.positive {{ color: #00ff41; }}
                .metric-value.negative {{ color: #ff4757; }}
                .metric-value.neutral {{ color: #ffa502; }}
                
                @media (max-width: 768px) {{
                    .metrics-grid, .charts-grid, .kpi-gauges-container {{ grid-template-columns: 1fr; }}
                }}
            </style>
        </head>
        <body>
            <div class="header">
                <h1>🚀 AlgoHaus Dark Report v5.0</h1>
                <p>📊 {pair} | 💡 {strategy.__name__} | ⏰ {timeframe} | 📅 {timestamp}</p>
            </div>
            
            <div class="kpi-dashboard-section">
                <h2>📊 Interactive KPI Dashboard</h2>
                {kpi_dashboard_html}
            </div>
            
            <div class="metrics-section">
                <h2 style="color: #00ff41; font-size: 18px; margin-bottom: 0;">📈 Performance Metrics</h2>
                {HTMLReportGenerator.generate_metrics_dashboard(metrics, initial_balance)}
            </div>
        </body>
        </html>
        """
        
        temp_dir = tempfile.gettempdir()
        report_path = os.path.join(temp_dir, f"dark_report_{pair.replace('/', '_')}_{strategy.__name__}_{timestamp}.html")
        
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write(html)
            
        return report_path
    
    @staticmethod
    def generate_metrics_dashboard(metrics, initial_balance):
        def get_metric_class(value, metric_type):
            if metric_type == 'return':
                return 'positive' if value >= 0 else 'negative'
            elif metric_type == 'winrate':
                return 'positive' if value >= 50 else 'negative'
            elif metric_type == 'factor':
                return 'positive' if value >= 1 else 'negative'
            else:
                return 'neutral'
        
        html = f"""
        <div class="metrics-grid">
            <div class="metric-card">
                <div class="metric-label">Initial Balance</div>
                <div class="metric-value neutral">${initial_balance:,.2f}</div>
            </div>
            <div class="metric-card">
                <div class="metric-label">Final Balance</div>
                <div class="metric-value {get_metric_class(metrics.get('final_balance_$', 0) - initial_balance, 'return')}">${metrics.get('final_balance_$', 0):,.2f}</div>
            </div>
            <div class="metric-card">
                <div class="metric-label">Total Return</div>
                <div class="metric-value {get_metric_class(metrics.get('total_return_%', 0), 'return')}">{metrics.get('total_return_%', 0):+.2f}%</div>
            </div>
            <div class="metric-card">
                <div class="metric-label">Win Rate</div>
                <div class="metric-value {get_metric_class(metrics.get('win_rate_%', 0), 'winrate')}">{metrics.get('win_rate_%', 0):.1f}%</div>
            </div>
            <div class="metric-card">
                <div class="metric-label">Profit Factor</div>
                <div class="metric-value {get_metric_class(metrics.get('profit_factor', 0), 'factor')}">{metrics.get('profit_factor', 0):.2f}</div>
            </div>
            <div class="metric-card">
                <div class="metric-label">Sharpe Ratio</div>
                <div class="metric-value neutral">{metrics.get('sharpe_ratio', 0):.2f}</div>
            </div>
        </div>
        """
        return html

# ======================================================================
# 7. DARK THEME UI
# ======================================================================
class BacktesterUI:
    def __init__(self, master):
        self.master = master
        master.title("🚀 AlgoHaus Backtester v5.0 - Complete Dark Edition")
        master.geometry("1400x900")
        master.minsize(1200, 700)
        
        self.data_folder = pathlib.Path.cwd() / "data" 
        self.df = None
        self.selected_pair = tk.StringVar(master, value="EUR/USD")
        self.selected_timeframe = tk.StringVar(master, value="1hr")
        self.selected_strategy = tk.StringVar(master, value="vwap_crossover_strategy")
        self.initial_balance = tk.DoubleVar(master, value=10000.0)
        self.unit_size = tk.IntVar(master, value=10000)
        self.leverage = tk.IntVar(master, value=50)
        self.sl_pips = tk.IntVar(master, value=30)
        self.tp_pips = tk.IntVar(master, value=60)
        
        today = date.today()
        self.end_date_var = tk.StringVar(master, value=today.strftime("%Y-%m-%d"))
        self.start_date_var = tk.StringVar(master, value=(today - timedelta(days=365)).strftime("%Y-%m-%d"))
        
        self.status_text = tk.StringVar(master, value="Ready - Complete Dark Edition")
        self.metrics_data = {}
        self.trades_df = pd.DataFrame()
        
        self.setup_ui()

    def setup_ui(self):
        main_container = ctk.CTkFrame(self.master, corner_radius=0)
        main_container.pack(fill='both', expand=True)
        
        left_panel = ctk.CTkFrame(main_container, corner_radius=15, width=450)
        left_panel.pack(side='left', fill='both', padx=(20, 10), pady=20)
        left_panel.pack_propagate(False)
        
        right_panel = ctk.CTkFrame(main_container, corner_radius=15)
        right_panel.pack(side='right', fill='both', expand=True, padx=(10, 20), pady=20)
        
        # Left Panel
        title_frame = ctk.CTkFrame(left_panel, corner_radius=10, fg_color="#1a1a1a")
        title_frame.pack(fill='x', padx=20, pady=(20, 15))
        
        title_label = ctk.CTkLabel(
            title_frame,
            text="⚡ ALGOHAUS DARK EDITION",
            font=ctk.CTkFont(family="Helvetica", size=18, weight="bold"),
            text_color="#00ff41"
        )
        title_label.pack(pady=15)
        
        controls_scroll = ctk.CTkScrollableFrame(left_panel, corner_radius=10, fg_color="transparent")
        controls_scroll.pack(fill='both', expand=True, padx=10, pady=(0, 10))
        
        # Configuration
        config_frame = ctk.CTkFrame(controls_scroll, corner_radius=10)
        config_frame.pack(fill='x', pady=(0, 15))
        
        config_title = ctk.CTkLabel(
            config_frame,
            text="⚙️ Configuration",
            font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
            text_color="#00ff41",
            anchor="w"
        )
        config_title.pack(fill='x', padx=15, pady=(10, 5))
        
        # Data folder
        folder_frame = ctk.CTkFrame(config_frame, fg_color="transparent")
        folder_frame.pack(fill='x', padx=15, pady=5)
        
        ctk.CTkLabel(folder_frame, text="Data Folder:", width=100).pack(side='left')
        
        self.folder_label = ctk.CTkLabel(
            folder_frame,
            text=str(self.data_folder)[:25] + "...",
            text_color="#888888",
            anchor="w"
        )
        self.folder_label.pack(side='left', expand=True, fill='x', padx=10)
        
        ctk.CTkButton(
            folder_frame,
            text="Browse",
            command=self.select_data_folder,
            width=70,
            height=28
        ).pack(side='right')
        
        # Input fields
        self.create_input_field(config_frame, "Trading Pair:", self.selected_pair, 
                               is_combobox=True, values=list(ForexCalculator.PIP_VALUES.keys()))
        self.create_input_field(config_frame, "Timeframe:", self.selected_timeframe,
                               is_combobox=True, values=["1min", "5min", "15min", "1hr", "1Day"])
        self.create_input_field(config_frame, "Start Date:", self.start_date_var)
        self.create_input_field(config_frame, "End Date:", self.end_date_var)
        
        # Strategy section
        strategy_frame = ctk.CTkFrame(controls_scroll, corner_radius=10)
        strategy_frame.pack(fill='x', pady=(0, 15))
        
        strategy_title = ctk.CTkLabel(
            strategy_frame,
            text="💡 Strategy & Risk",
            font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
            text_color="#00ff41",
            anchor="w"
        )
        strategy_title.pack(fill='x', padx=15, pady=(10, 5))
        
        strategies = [name for name, obj in inspect.getmembers(TradingStrategies) if inspect.isfunction(obj)]
        self.create_input_field(strategy_frame, "Strategy:", self.selected_strategy,
                               is_combobox=True, values=strategies)
        self.create_input_field(strategy_frame, "Stop Loss (Pips):", self.sl_pips)
        self.create_input_field(strategy_frame, "Take Profit (Pips):", self.tp_pips)
        
        # Account section
        account_frame = ctk.CTkFrame(controls_scroll, corner_radius=10)
        account_frame.pack(fill='x', pady=(0, 15))
        
        account_title = ctk.CTkLabel(
            account_frame,
            text="🏦 Account Settings",
            font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
            text_color="#00ff41",
            anchor="w"
        )
        account_title.pack(fill='x', padx=15, pady=(10, 5))
        
        self.create_input_field(account_frame, "Initial Balance ($):", self.initial_balance)
        self.create_input_field(account_frame, "Unit Size:", self.unit_size)
        self.create_input_field(account_frame, "Leverage:", self.leverage,
                               is_combobox=True, values=[str(x) for x in ForexCalculator.LEVERAGE_OPTIONS])
        
        # Run button
        run_button = ctk.CTkButton(
            left_panel,
            text="🚀 RUN BACKTEST",
            command=self.start_backtest_thread,
            height=45,
            corner_radius=8,
            font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
            fg_color="#00ff41",
            hover_color="#32ff7e",
            text_color="#000000"
        )
        run_button.pack(fill='x', padx=20, pady=(10, 20))
        
        # Right Panel
        summary_frame = ctk.CTkFrame(right_panel, corner_radius=10)
        summary_frame.pack(fill='x', padx=15, pady=(15, 10))
        
        summary_title = ctk.CTkLabel(
            summary_frame,
            text="📈 Summary",
            font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
            text_color="#00ff41",
            anchor="w"
        )
        summary_title.pack(fill='x', padx=15, pady=(10, 5))
        
        self.summary_textbox = ctk.CTkTextbox(
            summary_frame,
            height=150,
            corner_radius=8,
            font=ctk.CTkFont(family="Courier New", size=12),
            fg_color="#1a1a1a",
            text_color="#00ff41"
        )
        self.summary_textbox.pack(fill='both', padx=15, pady=(5, 15))
        
        # Metrics section
        metrics_frame = ctk.CTkFrame(right_panel, corner_radius=10)
        metrics_frame.pack(fill='both', expand=True, padx=15, pady=(0, 10))
        
        metrics_title = ctk.CTkLabel(
            metrics_frame,
            text="📊 Detailed Metrics",
            font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
            text_color="#00ff41",
            anchor="w"
        )
        metrics_title.pack(fill='x', padx=15, pady=(10, 5))
        
        self.metrics_scroll = ctk.CTkScrollableFrame(
            metrics_frame,
            corner_radius=8,
            fg_color="#1a1a1a"
        )
        self.metrics_scroll.pack(fill='both', expand=True, padx=15, pady=(5, 15))
        
        # Report button
        self.report_button = ctk.CTkButton(
            right_panel,
            text="📋 Generate Dark Report",
            command=self.generate_report,
            height=40,
            corner_radius=8,
            font=ctk.CTkFont(family="Helvetica", size=13, weight="bold"),
            fg_color="#434446",
            hover_color="#595C5E",
            state="disabled"
        )
        self.report_button.pack(fill='x', padx=15, pady=(0, 15))
        
        # Status bar
        status_frame = ctk.CTkFrame(self.master, corner_radius=0, height=35, fg_color="#111111")
        status_frame.pack(side='bottom', fill='x')
        
        self.status_label = ctk.CTkLabel(
            status_frame,
            textvariable=self.status_text,
            font=ctk.CTkFont(family="Helvetica", size=11),
            text_color="#888888",
            anchor="w"
        )
        self.status_label.pack(side='left', padx=20, pady=8)

    def create_input_field(self, parent, label_text, variable, is_combobox=False, values=None):
        frame = ctk.CTkFrame(parent, fg_color="transparent")
        frame.pack(fill='x', padx=15, pady=5)
        
        label = ctk.CTkLabel(frame, text=label_text, width=120, anchor="w")
        label.pack(side='left')
        
        if is_combobox:
            widget = ctk.CTkComboBox(frame, variable=variable, values=values or [])
        else:
            widget = ctk.CTkEntry(frame, textvariable=variable)
        widget.pack(side='left', expand=True, fill='x', padx=(10, 0))
        
        return widget

    def update_status(self, text, color='#888888'):
        self.status_text.set(text)
        self.status_label.configure(text_color=color)

    def select_data_folder(self):
        new_folder = filedialog.askdirectory(title="Select Data Folder (PARQUET files)")
        if new_folder:
            self.data_folder = pathlib.Path(new_folder)
            folder_text = str(self.data_folder)
            if len(folder_text) > 25:
                folder_text = "..." + folder_text[-22:]
            self.folder_label.configure(text=folder_text)
            self.update_status(f"Data folder set", '#00ff41')

    def start_backtest_thread(self):
        self.update_status("Starting backtest...", '#ffa502')
        
        self.summary_textbox.delete("0.0", "end")
        self.trades_df = pd.DataFrame()
        
        for widget in self.metrics_scroll.winfo_children():
            widget.destroy()
        
        self.report_button.configure(state="disabled")
        
        try:
            start_date = datetime.strptime(self.start_date_var.get(), "%Y-%m-%d")
            end_date = datetime.strptime(self.end_date_var.get(), "%Y-%m-%d")
            
            if start_date >= end_date:
                raise ValueError("Start date must be before End date.")

        except ValueError as e:
            messagebox.showerror("Input Error", f"Invalid input: {e}")
            self.update_status("Error: Invalid input.", '#ff4757')
            return

        self.q = queue.Queue()
        threading.Thread(target=self.run_backtest_task, 
                        args=(start_date, end_date), 
                        daemon=True).start()
        self.master.after(100, self.check_queue)

    def run_backtest_task(self, start_date, end_date):
        try:
            pair = self.selected_pair.get()
            timeframe = self.selected_timeframe.get()
            strategy_name = self.selected_strategy.get()
            
            df, _, _ = load_pair_data(pair, self.data_folder, start_date, end_date, timeframe)
            self.df = df
            
            strategy_func = getattr(TradingStrategies, strategy_name)
            
            backtester = EnhancedBacktester(
                df, 
                initial_balance=self.initial_balance.get(), 
                unit_size=self.unit_size.get(), 
                pip_value=ForexCalculator.PIP_VALUES.get(pair, 0.0001), 
                leverage=self.leverage.get()
            )
            
            summary, metrics = backtester.run_backtest(
                strategy_func, 
                self.sl_pips.get(), 
                self.tp_pips.get(), 
                pair
            )
            
            self.q.put(('success', summary, metrics, backtester.results))

        except Exception as e:
            self.q.put(('error', str(e)))

    def check_queue(self):
        try:
            result_type, *data = self.q.get_nowait()
            
            if result_type == 'success':
                summary, metrics, trades_df = data
                self.update_results_ui(summary, metrics, trades_df)
                self.update_status("✅ Backtest completed!", '#00ff41')
            elif result_type == 'error':
                error_msg = data[0]
                messagebox.showerror("Backtest Error", error_msg)
                self.update_status("❌ Error: Backtest failed.", '#ff4757')

        except queue.Empty:
            self.master.after(100, self.check_queue)

    def update_results_ui(self, summary, metrics, trades_df):
        self.trades_df = trades_df
        
        self.summary_textbox.delete("0.0", "end")
        self.summary_textbox.insert("0.0", summary)
        
        self.metrics_data = metrics
        
        for widget in self.metrics_scroll.winfo_children():
            widget.destroy()
        
        # Create metrics cards
        row_frame = None
        for i, (key, value) in enumerate(metrics.items()):
            if i % 2 == 0:
                row_frame = ctk.CTkFrame(self.metrics_scroll, fg_color="transparent")
                row_frame.pack(fill='x', pady=5)
            
            card = ctk.CTkFrame(row_frame, corner_radius=8, fg_color="#252525", width=250)
            card.pack(side='left', expand=True, fill='x', padx=5)
            card.pack_propagate(False)
            
            display_key = key.replace('_', ' ').title()
            
            if isinstance(value, (int, float)):
                if 'profit' in key.lower() or 'return' in key.lower():
                    color = '#00ff41' if value > 0 else '#ff4757'
                else:
                    color = '#00ff41'
            else:
                color = '#888888'
            
            label = ctk.CTkLabel(
                card,
                text=display_key,
                font=ctk.CTkFont(family="Helvetica", size=11),
                text_color="#888888",
                anchor="w"
            )
            label.pack(fill='x', padx=10, pady=(8, 2))
            
            value_str = f"{value:,.2f}" if isinstance(value, float) else str(value)
            value_label = ctk.CTkLabel(
                card,
                text=value_str,
                font=ctk.CTkFont(family="Helvetica", size=16, weight="bold"),
                text_color=color,
                anchor="w"
            )
            value_label.pack(fill='x', padx=10, pady=(2, 8))
        
        self.report_button.configure(state="normal")

    def generate_report(self):
        if self.trades_df.empty:
            messagebox.showwarning("Report Warning", "No trades found.")
            return

        try:
            self.update_status("Generating dark report...", '#ffa502')
            
            report_path = HTMLReportGenerator.generate_report(
                self.metrics_data, 
                self.trades_df, 
                getattr(TradingStrategies, self.selected_strategy.get()), 
                self.selected_timeframe.get(), 
                self.selected_pair.get(), 
                self.initial_balance.get(),
                self.unit_size.get(),
                df=self.df
            )
            
            messagebox.showinfo("Report Generated", f"Dark report saved:\n{report_path}")
            webbrowser.open_new_tab('file://' + os.path.realpath(report_path))
            self.update_status("📊 Dark report generated!", '#00ff41')
        
        except Exception as e:
            messagebox.showerror("Report Error", f"Failed to generate report: {e}")
            self.update_status("❌ Error generating report.", '#ff4757')

# ======================================================================
# 8. MAIN ENTRY POINT
# ======================================================================
if __name__ == '__main__':
    app = ctk.CTk()
    
    app.update_idletasks()
    width = 1400
    height = 900
    x = (app.winfo_screenwidth() // 2) - (width // 2)
    y = (app.winfo_screenheight() // 2) - (height // 2)
    app.geometry(f'{width}x{height}+{x}+{y}')
    
    backtester = BacktesterUI(app)
    app.mainloop()